In [2]:
# Code imports
from lib.opt_types import *
from lib.utils import *

# First Order Methods - 25 points

As a means of estimating the unknown parameter $\mathbf{x}^\natural$, your labor in part 1 led you to define the following optimization problem:
\begin{equation}
	\mathbf{x}^\star=\arg\min_{\mathbf{x} \in \mathbb{R}^p} f_\mu(\mathbf{x}) \equiv  f(\mathbf{x}) + \frac{\mu}{2}\|\mathbf{x}\|_2^2.
\tag{1}
\end{equation}

The optimality condition of (1) is

\begin{equation}
\nabla f_\mu({\mathbf x}^\star) = \sum_{i=1}^n -b_i \sigma(-b_i \cdot \mathbf{a}_i^T\mathbf{x}^\star)\mathbf{a}_i + \mu \mathbf{x}^\star = 0. \tag{2}
\end{equation}

Thanks to convexity, condition (2) is in fact the necessary and sufficient condition for $\mathbf x^\star$ to be optimal.
We can equivalently write this condition as
\begin{equation}
\mathbf x^\star = {\mathbf x}^\star - \mathbf{B} \nabla f_\mu({\mathbf x}^\star) \tag{3}
\end{equation}
for any symmetric, positive definite matrix $\mathbf{B}$. 
This is a fixed-point formulation of (1), which will be used to develop first-order methods.


Notice that choosing $\mathbf{B} = \alpha \mathbb{I}$ with $\alpha > 0$ in the formulation (3) suggests that the gradient descent method can be used to solve (1). 

In this part, you will implement different variants of the gradient descent algorithm. 


### Code structure:

A `Function` is a type that has the following attributes:

```python
@dataclass
class Function:
    f              # The function that is being minimized
    grad           # The gradient of the function
    f_star         # The minimum of the function
    strng_cvx      # The strong convexity constant, if it admits one
    lips_grad      # The smoothness constant, if it admits one
```

For a given function `f = Function()`:
- You can evaluate the function at a point `x` by simply writing `f(x)`.
- You can evaluate the gradient at a point `x` by writing `f.grad(x)`.
- You can get the smoothness constant by writing `f.lips_grad`.
- You can get the strong convexity constant by writing `f.strng_cvx`



We have defined the function that we are trying to minimize $f_\mu$ and stored in the object `f`, and we have provided you with an inital guess of a solution `x_zero`.

In [3]:
from lib.part_one import f, x_zero

In order for you to test your algorithms, we have also provided you with a `test` function that runs a given method on a strongly convex quadratic function $x \mapsto \frac{1}{2}x^\top M x$ for `100` iterations that we intialize at $x_0 = \begin{bmatrix} 100.0& 100.0 \end{bmatrix}$. For example, once you implement `GD` you can test it by running

```python
test(GD)
```

<div class="alert alert-block alert-success">
<b>Run the cells !</b> Make sure you run all the code cells as some variables are defined and used in mutiple subsequent cells.
</div>

### Variants of Gradient Descent

Now you will implement various methods to iteratively improve the initial guess `x_zero`.

__(a)__ (2.5 points) The main ingredients of the gradient descent algorithm are the descent direction (the negative gradient) and the step-size $\alpha_k$. 
In this part, we consider the gradient descent algorithm with constant step-size ${1}/{L}$, i.e., $\alpha_k = {1}/{L}$ for any $k = 0,1,...$ where $L$ is the gradient-Lipschitz constant of the function `f`. 

Gradient Descent, as an iterative scheme, only tracks the current iterate `x_k` and the step size `alpha_k`. We store them both in a state variable that holds both these attributes:

In [4]:
@dataclass
class GD_state(OptState):
    x_k: Vector
    alpha_k: float

Given a function `f` and a state at some iteration $k$, recall how GD updates the state and fill in the following cell:

In [1]:
def GD_update(f, state):
    x_k, alpha_k = state
    
    x_next = #FILL
    
    return GD_state(x_k = x_next, alpha_k = alpha_k)

Having defined the `GD_update` which updates a given state, we now need to initialize the state. Fill in the cell below, recalling that you are given an initial guess `x_zero`, and the fact that we are using a fixed step size corresponding to the smoothness constant of `f`.

In [ ]:
def GD_initialize(f, x_zero):
    x_k = #FILL
    alpha_k = #FILL
    return GD_state(x_k, alpha_k)

With the initialization and the update scheme specified, we define the algorithm in the cell below:

In [ ]:
GD = OptAlgorithm(name="GD", init_state = GD_initialize, state_update = GD_update)

In [ ]:
test(GD)

We now notice that the objective function is strongly convex. Therefore, we can select the constant step-size $\alpha$ as $2/(L+\mu)$ to get a faster convergence rate. We call this algorithm `GDstr`.

Both `GD` and `GDstr` have the same state update but only differ in the stepsize they use. Implement this variant of the gradient descent method by completing the code:

In [ ]:
def GDstr_initialize(f, x_zero):
    x_k = #FILL
    alpha_k = #FILL
    return GD_state(x_k, alpha_k)
    
GDstr = OptAlgorithm(name="GDstr", init_state = GDstr_initialize, state_update = GD_update)

In [ ]:
test(GDstr)

__(b-1)__ (1.5 points) We can accelerate the above gradient descent algorithm with the following scheme ($t_0=1$):

\begin{equation*}
\begin{array}{ll}
\mathbf{x}^{k+1} &:=  \mathbf{y}^k - \alpha_k\nabla{f_\mu}(\mathbf{y}^k),\\
t_{k+1} &:= \frac{1}{2}(1 + \sqrt{1 + 4t_k^2})\\
\mathbf{y}^{k+1} &:= \mathbf{x}^{k+1} + \frac{t_k-1}{t_{k+1}}\big(\mathbf{x}^{k+1} - \mathbf{x}^k\big).
\end{array}
\end{equation*}

Implement this algorithm with constant step-size $\alpha = 1/L$, by completing the missing parts of the following cells.

First we define the state that the algorithm updates at each iteration:

In [ ]:
@dataclass
class AGD_state(OptState):
    x_k: Vector
    y_k: Vector
    t_k: float
    alpha_k: float

Given this `state` and a function `f` to optimize, write the update step of accelerated gradient descent:

In [ ]:
def AGD_update(f, state):
    x_k, y_k, t_k, alpha_k = state
    
    next_x_k = #FILL
    next_t_k = #FILL
    next_y_k = #FILL
    next_alpha_k = #FILL
    
    return AGD_state(x_k = next_x_k, y_k = next_y_k, t_k=next_t_k, alpha_k =next_alpha_k)

Define the initial state of AGD in the cell below:

In [ ]:
def AGD_initialize(f, x_zero):
    return AGD_state(x_k = x_zero, y_k = x_zero, t_k=1, alpha_k =1.0/f.lips_grad)

We have all the necessary ingredients to define the optimization algorithm in the following cell:

In [ ]:
AGD = OptAlgorithm(name="AGD", init_state=AGD_initialize, state_update=AGD_update)

In [ ]:
test(AGD)

__(b-2)__ (1 points)

Note that the objective function is strongly convex. Therefore, we can use the accelerated gradient algorithm for strongly convex objectives to converge faster. This variant can be summarized as follows:

\begin{equation*}
\left\{\begin{array}{ll}
\mathbf{x}^{k+1} &:=  \mathbf{y}^k - \alpha_k\nabla{f_\mu}(\mathbf{y}^k),\\
\mathbf{y}^{k+1} &:= \mathbf{x}^{k+1} + m\big(\mathbf{x}^{k+1} - \mathbf{x}^k\big).
\end{array}\right. 
\end{equation*}
 
Implement this variant of the accelerated gradient method by completing the following cells. Try different values of $m$ and comment on what should be the best value for this constant. Do the practical results follow the theory?

In [ ]:
@dataclass
class AGDstr_state(OptState):
    x_k: Vector
    y_k: Vector
    alpha_k: float
    m_strongcvx: float

In [ ]:
def AGDstr_update(f, state):
    x_k, y_k, alpha_k, m_strongcvx = state
    
    #FILL
    
    return AGDstr_state(next_x_k, next_y_k, alpha_k, m_strongcvx)

def AGDstr_initialize(f, x_zero):
    m_strongcvx = ... # Pick value of m here
    #FILL

In [ ]:
AGDstr = OptAlgorithm(name="AGDstr", init_state=AGDstr_initialize, state_update=AGDstr_update)

In [ ]:
test(AGDstr)

__(c)__ (5 points) We can obtain better performance by considering a line search procedure, which adapts the step-size $\alpha_k$ to the local geometry. 
The line-search strategy to determine the step-size $\alpha_k$ for the standard GD algorithm,
$\mathbf{x}^{k+1}=\mathbf{x}^k-\alpha_k\nabla f_\mu(\mathbf{x}^k),$
is the following: 

At the step $kth$ iteration, let $\mathbf{x}^k$ be the current iterate and $\mathbf{d}^k=-\nabla f_\mu(\mathbf{x}^k)$ be a given descent direction, and perform:
* Set $L_0 = L$.
* At each iteration, set $L_{k,0} = \frac{1}{2}L_{k-1}$, where $k$ is the iteration counter. 
* Using a for loop, find the minimum integer $i \geq 0$ that satisfies $f_\mu\left(\mathbf{x}^k + \frac{1}{2^i L_{k,0}} \mathbf{d}^k\right) \leq f_\mu(\mathbf{x}^k) -\frac{1}{2^{i+1} L_{k,0}} \|\mathbf{d}^k\|^2$.
* Set $L_k = {2^i L_{k,0}}$ and use the step-size $\alpha_k := \frac{1}{L_k}$ (i.e., use the new estimate that you have used in the line-search: $\mathbf{x}^k + \frac{1}{2^i L_{k,0}} \mathbf{d}^k$).

Complete the missing parts in order to implement gradient descent with line-search.

In [ ]:
@dataclass
class LS_GD_state(OptState):
    #FILL

In [ ]:
def LS_GD_update(f, state):
    
    #FILL
    
    return #FILL
    
def LS_GD_initialize(f, x_zero):
    return #FILL

In [ ]:
LS_GD = OptAlgorithm(name="LS_GD", init_state = LS_GD_initialize, state_update=LS_GD_update )

In [ ]:
test(LS_GD)

We now incorporate a line-search enhancement to the accelerated gradient method implemented in (b). At step $k$, 
we have the current iteration $\mathbf{x}^k$ together with an _intermediate variable_ $\mathbf{y}^k$ and its corresponding direction $\mathbf{d}^k=-\nabla f_\mu(\mathbf{y}^k)$. **Note that the intermediate variable is then used in the gradient step and hence the line-search will be performed on it to determine the step-size**.


Perform a line-search strategy with respect to $\mathbf{y}^k$ and direction $\mathbf{d}^k$ to determine the step-size $\alpha_k$ as follows:

* Set $L_0 = L$.
* At each iteration, set $L_{k,0} = \frac{1}{2}L_{k-1}$, where $k$ is the iteration counter. 
* Using a for loop, find the minimum integer $i \geq 0$ that satisfies $f_\mu\left(\mathbf{y}^k + \frac{1}{2^i L_{k,0}} \mathbf{d}^k\right) \leq f_\mu(\mathbf{y}^k) -\frac{1}{2^{i+1} L_{k,0}} \|\mathbf{d}^k\|^2$.
* Set $L_k = {2^i L_{k,0}}$ and use the step-size $\alpha_k := \frac{1}{L_k}$.
 
Update the next iterations:
\begin{equation*}
\left\{\begin{array}{ll}
\mathbf{x}^{k+1} &:=  \mathbf{y}^k - \alpha_k\nabla{f_\mu}(\mathbf{y}^k),\\
t_{k+1} &:= \frac{1}{2}\left(1 + \sqrt{1 + 4\frac{L_k}{L_{k-1}}t_k^2}\right)\\
\mathbf{y}^{k+1} &:= \mathbf{x}^{k+1} + \frac{t_k-1}{t_{k+1}}\big(\mathbf{x}^{k+1} - \mathbf{x}^k\big).
\end{array}\right.
\end{equation*}

Complete the missing parts in order to implement accelerated gradient descent with line-search.

In [ ]:
@dataclass
class LS_AGD_state(OptState):
    #FILL

In [ ]:
def LS_AGD_update(f, state):
    #FILL
    
    return #FILL

def LS_AGD_initialize(f, x_zero):
    return #FILL


In [ ]:
LS_AGD = OptAlgorithm(name="LS_AGD", init_state=LS_AGD_initialize, state_update=LS_AGD_update )#FILL

In [ ]:
test(LS_AGD)

__(d)__ (5 points) The accelerated gradient method is non-monotonic, so it can be oscillatory, i.e. $f_\mu(\mathbf{x}^{k+1})\not\leq f_\mu(\mathbf{x}^k)$ for all $k\geq 0$.
To prevent such behavior, we can use the so-called adaptive restart strategy. 
In short, this strategy can be explained as follows: 
At each iteration, whenever $\mathbf{x}^{k+1}$ is computed, we evaluate $f_\mu(\mathbf{x}^{k+1})$ and compare it with $f_\mu(\mathbf{x}^k)$: 

* If $f_\mu(\mathbf{x}^k) < f_\mu(\mathbf{x}^{k+1})$, restart the iteration, i.e., recompute $\mathbf{x}^{k+1}$ by setting $\mathbf{y}^{k} := \mathbf{x}^{k}$ and $t_{k} := 1$;
* Otherwise, let the algorithm continue.


This strategy requires the evaluation of the function value at each iteration, which increases the computational complexity of the overall algorithm. 

Implement the adaptive restart strategy which uses the function values for the accelerated gradient algorithm with constant step-size $\alpha_k=1/L$ 
by completing the function:

In [ ]:
@dataclass
class AGDR_state(AGD_state):
    #FILL

In [ ]:
def AGDR_update(f, state):
    #FILL
    
    return #FILL

AGDR_initialize = AGD_initialize

In [ ]:
AGDR = OptAlgorithm(name="AGDR", init_state = AGDR_initialize, state_update=AGDR_update)

In [ ]:
test(AGDR)

__(e)__ (Self-study, 0 points)
Incorporate the line-search, acceleration and function values restart by completing the function LSAGDR. 

**Hint:** Note that while the restart is executed on $\mathbf{x}^k$, the line-search strategy is executed on $\mathbf{y}^k$ and the direction $\mathbf{d}^k=-\nabla f(\mathbf{y}^k)$ to determine the 
step-size and hence, use line-search each time you encounter an intermediate variable $\mathbf{y}^k$.

In [ ]:
@dataclass 
class LS_AGDR_state(LS_AGD_state):
    #FILL

In [ ]:
def LS_AGDR_update(f, state):
    #FILL
    
    return #FILL

def LS_AGDR_initialize(f, x_zero):
    return #FILL

In [ ]:
LS_AGDR = OptAlgorithm(name="LS_AGDR", init_state=LS_AGDR_initialize, state_update=LS_AGDR_update)

test(LS_AGDR)

__(f)__ (4 points) We can also apply an optimization technique that does not exploit the knowledge of the Lipschitz constant, and instead adapts to the local geometry by making use of past gradient information. AdaGrad adapts the step-size using the inverse square $\ell_2$-norm of past gradients. Starting with $Q_0 = 0$ it iterates as follows:

\begin{equation*}
\left\{\begin{array}{ll}
Q_k &{\!\!\!\!}= Q_{k-1} + \|\nabla f_\mu(\mathbf{x}^k)\|^2 \\
\mathbf{H}_k &{\!\!\!\!}= (\sqrt{Q_k} + \delta) I \\
\mathbf{x}^{k+1} &{\!\!\!\!}= \mathbf{x}^k - \alpha \mathbf{H}_k^{-1} \nabla f_\mu(\mathbf{x}^k)
\end{array}\right.
\end{equation*}

Complete the missing parts in the function AdaGrad in order to implement the above adaptive gradient method using $\alpha = 1$, $\delta = 10^{-5}$.

In [ ]:
@dataclass
class AdaGrad_state(OptState):
    #FILL

In [ ]:
def AdaGrad_update(f, state):
    #FILL
    
    return #FILL

def AdaGrad_initialize(f, x_zero):
    return #FILL

In [ ]:
AdaGrad = OptAlgorithm(name="AdaGrad", init_state=AdaGrad_initialize, state_update=AdaGrad_update)
test(AdaGrad, maxiter=30000)

__(g)__ (2.5 points) Another famous adaptive optimization method is called the ADAptive Moment estimation algorithm, also known as ADAM, 

\begin{equation*}
\left\{\begin{array}{ll}
\mathbf{g}_k &{\!\!\!\!} = \nabla f_\mu(\mathbf{x}^{k}) \\
\mathbf{m}_{k+1} &{\!\!\!\!} = \beta_1 \mathbf{m}_{k} + (1-\beta_1)\mathbf{g}_k \ \textrm{$\leftarrow$ Momentum}\\
\mathbf{v}_{k+1} &{\!\!\!\!} = \beta_2 \mathbf{v}_{k} + (1-\beta_2)\mathbf{g}_k^2 \ \textrm{$\leftarrow$ Adaptive term}\\
\hat{\mathbf{m}}_{k+1} &{\!\!\!\!} = \mathbf{m}_{k+1} / (1-\beta_1^{k+1})\\
\hat{\mathbf{v}}_{k+1} &{\!\!\!\!} = \mathbf{v}_{k+1} / (1-\beta_2^{k+1}) \ \textrm{$\leftarrow$ Scaling for removing bias}\\
\mathbf{H}_{k+1} &{\!\!\!\!} \ = \sqrt{\hat{\mathbf{v}}_{k+1}} + \epsilon \\
\mathbf{x}^{k+1} &{\!\!\!\!} = \mathbf{x}^k - \alpha \hat{\mathbf{m}}_{k+1} / \mathbf{H}_{k+1}
\end{array}\right.
\end{equation*}

Note that all operations shown above, when applied to vectors, are applied element-wise. In particular, $\mathbf{g}_k^2$ is a vector of the same size as $\mathbf{g}_k$ where each element is squared.

Complete the missing parts in the cells below in order to implement the above adaptive gradient method using $\alpha = 0.1$, $\beta_1 = 0.9$, $\beta_2 = 0.999$ and $\epsilon = 10^{-8}$.

It has been shown that ADAM can fail to converge to the global minimum of a convex problem. The authors provided a variant of ADAM, called AMSgrad in order to fix this convergence issue. However, in practice it is not clear which method performs best. (You are not required to implement this method, but advised to have a look at it for personal interest).

In [ ]:
@dataclass
class ADAM_state(OptState):
    #FILL

In [ ]:
def ADAM_update(f, state):
    #FILL
    
    return #FILL

def ADAM_initialize(f, x_zero):
    return #FILL
    

In [ ]:
ADAM = OptAlgorithm(name="ADAM", init_state=ADAM_initialize, state_update=ADAM_update)

test(ADAM, maxiter=1000)

## Scheduler Free Optimizer

Schedule-Free learning replaces the momentum of an underlying optimizer with a combination of interpolation and averaging. In the case of gradient descent, the basic Schedule-Free update is:

$$
\begin{align*}
y_{t} & = (1-\beta)z_{t} + \beta x_{t},\\
z_{t+1} & =z_{t}-\gamma\nabla f(y_{t}),\\
x_{t+1} & =\left(1-\frac{1}{t+1}\right)x_{t}+\frac{1}{t+1}z_{t+1},
\end{align*}
$$

Here $x$ is the sequence that evaluations of test/val loss should occur at, which differs from the primary iterates $z$ and the gradient evaluation locations $y$. The updates to $z$ correspond to the underlying optimizer, in this case a simple gradient step.

As the name suggests, Schedule-Free learning does not require a decreasing learning rate schedule, yet typically out-performs, or at worst matches, SOTA schedules such as cosine-decay and linear decay. Only two sequences need to be stored at a time (the third can be computed from the other two on the fly) so this method has the same memory requirements as the base optimizer (parameter buffer + momentum).




In [6]:
@dataclass
class Scheduler_free_state(OptState):
    #FILL


**(a)** (0.5 points) Fill the initializer for the scheduler free optimizer.


In [7]:
def Scheduler_free_update(f, state):
    #FILL
    
    return #FILL

def Scheduler_free_initialize(f, x_zero):
    return #FILL


**(b)** (1 points) Build the scheduler free optimizer main update state.

In [ ]:
SFO = OptAlgorithm(name="Scheduler_Free_Optimizer", init_state=Scheduler_free_initialize, state_update=Scheduler_free_update)

test(SFO, maxiter=1000)

**(c)** (1 points) Run your experiment and discuss why the algorithm is referred to as a scheduler-free optimizer. For reference, it is beneficial to review [Road Less Scheduled](https://arxiv.org/abs/2405.15682)
 to gain a deeper understanding of the concepts and rationale behind this terminology.
 

## Running the methods - 1 Point


With these iterative shemes implemented, we will try to solve one logistic regression problem.

We are going to model the probablity that a basketball shot succesfully enters the net using the logistic model introduced in the first part of the homework.

Our data originates from motion tracking data captured by the NBA during the 2015-2016 season.

<img width=50% src='lib/golden_state_shots.png'/>

We consider the `3376` shots made by the Golden State Warriors. The labels ${b}_i$ will correspond to whether or not the shot $i$ was succesful, and $\mathbf{a}_i$ will be a feature vector of dimension 5 containing the x-location, the y-location (of the player shooting on the basketball court), the distance to the basket, the number of minutes and the number of seconds left before the end of the quarter.

The following cell runs the methods you have implemented on the logistic loss:

In [ ]:
from lib.part_one import f, x_zero, train_accuracy

#Remove methods you have not implemented
methods = [GD, GDstr, AGD, AGDstr, LS_GD, AGDR, AdaGrad, ADAM, SFO]

plot(methods, f, x_zero, max_iteration=5000)

Now, think you are an NBA team general manager and you have access to the data described above for the 2015-2016 season. You want to pick a good performing method to predict the scoring efficiency of opposite teams - in order to adjust your team accordingly. Run the fillowing cell to display the training accuracy of your solution:

In [ ]:
method = #FILL
run_trace = run(method, f, x_zero, 5000)
last_x_k = run_trace.sequence[-1]

train_accuracy(run_trace.sequence[-1])

Given we have access to as many as `84467` distinct shots from all teams that played in 2015-2016, what could we do to improve our method? What consequences could that bring when running our deterministic methods? 